In [30]:
import pandas as pd

import numpy as np

import pymongo

In [31]:
MONGODB_URL = "mongodb://35.237.232.73:27017/"

client = pymongo.MongoClient(MONGODB_URL)

In [32]:
df = pd.read_csv("mpm_serventias.csv", sep=';', low_memory=False)
df.shape

(39100, 17)

In [33]:
df['SEQ_ORGAO'] = df['SEQ_ORGAO'].astype('int')
df['SEQ_ORGAO_PAI'] = df['SEQ_ORGAO_PAI'].fillna(0).astype('int')
df['DSC_ORGAO'] = df['DSC_ORGAO'].fillna("").astype('object')

In [34]:
df.head()

,SEQ_ORGAO,NOMEDAVARA,SEQ_ORGAO_PAI,TIP_ORGAO,SEQ_CIDADE,DSC_CIDADE,SIG_UF,COD_IBGE,DSC_TIP_ORGAO,TIP_ESFERA_JUSTICA,INT_ORDEM_ORGAO,DSC_ORGAO,DSC_DENOM_SERVENTIA_JUDICIAL,endereco_serventia,cep_serventia,LATITUDE,LONGITUDE
0,1,Conselho Nacional de Justiça,0,CNJ,743.0,BRASÍLIA,DF,5300108.0,Conselho Nacional de Justiça,S,0.0,Conselho Nacional de Justiça,NaN,NaN,NaN,NaN,NaN
1,2,Conselho da Justiça Federal,1,CONSF,743.0,BRASÍLIA,DF,5300108.0,Conselho da Justiça Federal,F,0.0,Conselho da Justiça Federal,NaN,NaN,NaN,NaN,NaN
2,3,Tribunal Regional Federal da 1ª Região,2,TRIBF,743.0,BRASÍLIA,DF,5300108.0,Tribunal Regional Federal,F,NaN,Tribunal Regional Federal da 1ª Região,NaN,NaN,NaN,NaN,NaN
3,4,Tribunal Regional Federal da 2ª Região,2,TRIBF,4155.0,RIO DE JANEIRO,RJ,3304557.0,Tribunal Regional Federal,F,NaN,Tribunal Regional Federal da 2ª Região,NaN,NaN,NaN,NaN,NaN
4,5,Tribunal Regional Federal da 3ª Região,2,TRIBF,4795.0,SAO PAULO,SP,3550308.0,Tribunal Regional Federal,F,NaN,Tribunal Regional Federal da 3ª Região,NaN,NaN,NaN,1050182067,607162856


In [35]:
def calcular_entrancia(seq_orgao, df_param):
  """
  Filtra as varas.
  Campo "dadosBasicos.orgaoJulgador.codigoOrgao".
  """
  df_f_orgao = df_param[df_param["SEQ_ORGAO"] == seq_orgao]

  if len(df_f_orgao) == 0:
    print(f"Orgão {seq_orgao} não encontrado!")
    return 0

  seq_orgao_pai = df_f_orgao['SEQ_ORGAO_PAI'].values[0]

  """
  Filtra as varas que tem o mesmo orgão pai
  """
  df_f_orgao_pai = df_param[df_param["SEQ_ORGAO_PAI"] == seq_orgao_pai]

  if len(df_f_orgao_pai) == 0:
    print(f"Orgão pai {seq_orgao_pai} não encontrado!")
    return 0

  total = df_f_orgao_pai["SEQ_ORGAO_PAI"].notnull().count()

  if total == 0:
    print(f"Orgao {seq_orgao} não possui orgão pai!")
    return 0

  if total == 1:
    return 1
  elif total in [2, 3, 4]:
    return 2
  else:
    return 3

In [44]:
def inserir_entrancia(database_name, collection_name, document, entrancia):
  
  collection = client[database_name][collection_name]

  _id =  document["_id"]
  
  collection.update_one(
      {
          "_id": _id 
      }, 
      {
          '$set': { 'dadosBasicos.orgaoJulgador.entrancia': entrancia }
      }
  )

In [45]:
def processar_database(database_name, filter_collection=None):
  print(f"Processando database: {database_name}")

  collections = client[database_name].list_collection_names()

  if filter_collection:
    collections = [ collection for collection in collections if collection in filter_collection]
  
  for collection_name in collections:
    documents = list(client[database_name][collection_name].find())

    print(f"Collection {collection_name} : total {len(documents)}")

    for document in documents:
        
      orgao_julgador = document["dadosBasicos"]["orgaoJulgador"]
      codigo_orgao = int(orgao_julgador["codigoOrgao"])

      entrancia = calcular_entrancia(codigo_orgao, df)

      if entrancia == 0:
        print("Não foi possível calcular a entrância:", orgao_julgador)

      inserir_entrancia(database_name, collection_name, document, entrancia)

In [46]:
processar_database("justica_estadual", ["processos-tjsp"])

Processando database: justica_estadual


KeyboardInterrupt: 

In [ ]:
database_para_processar = ['justica_eleitoral',
                           'justica_federal', 
                           'justica_militar', 
                           'justica_trabalho',
                           'justica_estadual',
                           'tribunais_superiores']